#  Inferens 

### Econometrics A (ØkA)

Wooldridge (Ch. 3)

Bertel Schjerning

Department of Economics, University of Copenhagen


### Enable autoreload

In [1]:
# Sørger for at alle importerede python filer geninlæses ved import statements
# Nødvedigt, hvis ændinger skal tage effekt uden at genstarte Python Kernel
%load_ext autoreload
%autoreload 2

# Part 1: t-test, p-værdier og konfidensintervaller

### Lønregression: Timeløn, uddannelse og erfaring
- Vi estimerer en lineær model for timeløn, uddannelse og erfaring:
$$
		\log(\text{wage}_i) = \beta _{0}+\beta _{1}\text{educ}_i+\beta_{2}\text{experience}_i+\beta _{3}\text{experience}^2_i+\beta _{4}\text{experience}^3_i+u_i
$$
- Vi ønsker f.eks. at teste hypotesen $H_0$ mod det dobbelt sidede alternativ $H_1: \beta_4 \neq 0$

$$H_0: \beta_4=0 \quad H_1: \beta_4 \neq 0$$

Test størrelse
$$t=\frac{\hat{\beta}_4-0}{\sqrt{Var(\hat{\beta}_4)}} \sim t(n-k)$$

### Indlæs data, estimer model og print resulater 

In [12]:
import pandas as pd
import numpy as np
import mymlr as mlr # see mymlr.py

# Load the data and create variables
df = pd.read_stata(r"../data/wage.dta")   # Load data
df['const'] = 1                           # Add constant term
df['lwage'] = np.log(df['wage'])          # Log of wage
df['experience2'] = df['experience'] ** 2 # Add experience²
df['experience3'] = df['experience'] ** 3 /1000 # Add experience^3 (divided by 1000)

# Estimate the model using the mlr.ols function
rhs=['const', 'educ', 'experience', 'experience2','experience3'] # Variables in the model
reg1 = mlr.ols(df[rhs], df['lwage']) # estimate model using mlr
mlr.output(reg1) # Print the summary using the mlr.summary() function


OLS Regression Results for Dependent Variable: lwage
Number of Observations: 1078
Degrees of Freedom: 1073 (Residual), 5 (Model)
R-squared: 0.2058
TSS: 111.2507, RSS: 88.3507, ESS: 22.9000
Variable                Coefficient     Std. Error           t       P>|t|    95% Conf. Interval
--------------------------------------------------------------------------------------------------
const                        4.2699         0.0466     91.7066      0.0000   [  4.1786, 4.3613  ]
educ                         0.0270         0.0027     10.1735      0.0000   [  0.0218, 0.0322  ]
experience                   0.0428         0.0102      4.1983      0.0000   [  0.0228, 0.0628  ]
experience2                 -0.0018         0.0007     -2.3782      0.0176   [ -0.0032, -0.0003 ]
experience3                  0.0286         0.0154      1.8543      0.0640   [ -0.0017, 0.0588  ]


### Beregn test størrelser, p-værdier og konfidensintervaller
**Er $t$ større end kritisk værdi for $\beta_4$? Afvises $H_0$ til fordel for $H_1$?**

In [3]:
from scipy import stats
t_stat = reg1['beta_hat'] / reg1['standard_errors']  # t-statistics: β / se(β)
n= reg1['n'] # Number of observations
k= reg1['k'] # Number of regressors (including constant)
alpha = 0.05 # Significance level
t_crit = stats.t.ppf(1-alpha/2, df=n - k) # Critical t-value for two-tailed test
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=n - k)) # Two-tailed p-values for t-stats
CI = np.array([reg1['beta_hat'] - t_crit * reg1['standard_errors'], 
               reg1['beta_hat'] + t_crit * reg1['standard_errors']]) # Confidence intervals

# Create a DataFrame to store results nicely without confidence intervals
output_df = pd.DataFrame({'Variable': reg1['lbl_X'],'Coefficient': reg1['beta_hat'].flatten(),'Std. Error': 
                          reg1['standard_errors'].flatten(),'t-stat': t_stat.flatten(),'P>|t|': p_values.flatten(), 'CI_low': CI[0].flatten(), 'CI_high': CI[1].flatten()
                          })

# Round values for better readability 
output_df=output_df.round(4)
# Display the DataFrame without the row index
print(output_df.to_string(index=False))
print(f"Critial t-value for 95% for to sidet test: {t_crit:.4f}")

   Variable  Coefficient  Std. Error  t-stat  P>|t|  CI_low  CI_high
      const       4.2699      0.0466 91.7066 0.0000  4.1786   4.3613
       educ       0.0270      0.0027 10.1735 0.0000  0.0218   0.0322
 experience       0.0428      0.0102  4.1983 0.0000  0.0228   0.0628
experience2      -0.0018      0.0007 -2.3782 0.0176 -0.0032  -0.0003
experience3       0.0286      0.0154  1.8543 0.0640 -0.0017   0.0588
Critial t-value for 95% for to sidet test: 1.9622


# Part 2: Returns to scale in French manufacturing
Hypotese test med lineære restriktioner på en eller flere parametre

### Estimation af Cobb-Douglas produktionsfunktion
- Vi estimerer en Cobb-Douglas produktionsfunktion for fransk fremstillingsindustri.
- Datasættet dækker N = 441 virksomheder i perioden 1968–79.
- Variable:
    - ldsa (log af deflateret omsætning, $\log(y_{it})$), 
    - lemp (log af beskæftigelse, $\log(L_{it})$), 
    - lcap (log af justeret kapitalbeholdning, $\log(K_{it})$)
    
    Vi vil behandle deflateret omsætning som produktionsoutput og estimere Cobb-Douglas formen for produktionen, hvilket fører til følgende sammenhæng:

$$ \log(Y_{it}) = a_0 + \alpha \log(L_{it}) + \beta \log(K_{it})  + u_{it} $$



In [4]:
dat=pd.read_csv('data/firms.csv') # Load data
dat['const'] = 1 # Add constant term
pd.options.display.float_format = '{:.4f}'.format # format output to 4 decimal points
dat.describe()

,firmid,year,lcap,lemp,ldsa,const
count,5292.0000,5292.0000,5292.0000,5292.0000,5292.0000,5292.0000
mean,221.0000,1973.5000,-0.0000,-0.0000,0.0000,1.0000
std,127.3174,3.4524,1.3110,1.1801,1.2325,0.0000
min,1.0000,1968.0000,-3.8649,-3.3828,-3.5515,1.0000
25%,111.0000,1970.7500,-0.9083,-0.7855,-0.9280,1.0000
50%,221.0000,1973.5000,-0.1181,-0.1137,-0.1030,1.0000
75%,331.0000,1976.2500,0.9063,0.7930,0.8562,1.0000
max,441.0000,1979.0000,4.1037,3.3713,3.9134,1.0000


### Estimer model og print resulater

In [5]:
import mymlr as mlr # see mymlr.py
CobbDouglas1 = mlr.ols(dat[['const', 'lcap', 'lemp']], dat['ldsa']) # estimate model using mlr
mlr.output(CobbDouglas1) # Print the summary using the mlr.summary() function

OLS Regression Results for Dependent Variable: ldsa
Number of Observations: 5292
Degrees of Freedom: 5289 (Residual), 3 (Model)
R-squared: 0.9139
TSS: 8037.3115, RSS: 691.9438, ESS: 7345.3677
Variable                Coefficient     Std. Error           t       P>|t|    95% Conf. Interval
--------------------------------------------------------------------------------------------------
const                        0.0000         0.0050      0.0000      1.0000   [ -0.0097, 0.0097  ]
lcap                         0.3100         0.0091     33.9237      0.0000   [  0.2921, 0.3280  ]
lemp                         0.6748         0.0102     66.4625      0.0000   [  0.6549, 0.6947  ]


### Test for constant returns to scale
Model
$$
\log(Y_{it}) = a_0 + \alpha \log(L_{it}) + \beta \log(K_{it})  + u_{it} 
$$

Vi tester hypothesen (lineær restriktion på flere parametre)
$$H_0: \alpha + \beta = 1 \quad H_1: \alpha + \beta \neq 1$$

Test størrelse
$$t=\frac{\hat{\alpha}+\hat{\beta}-1}{se(\hat{\alpha}+\hat{\beta})}=\frac{\hat{\alpha}+\hat{\beta}-1}{\sqrt{Var(\hat{\alpha})+Var(\hat{\beta})+2Cov(\hat{\alpha},\hat{\beta})}} \sim t(n-k)$$

### Let's code

In [6]:
# t-test for constant returns to scale (test of H0: α + β  = 1)
num=CobbDouglas1['beta_hat'][1]+CobbDouglas1['beta_hat'][2] - 1  # α + β -1
var_beta_hat=CobbDouglas1['var_beta_hat'] # Variance-covariance matrix of beta_hat
denum=np.sqrt(var_beta_hat[1,1]+var_beta_hat[2,2]+2*var_beta_hat[1,2]) # Standard error of α + β 
t_stat=num/denum # t-statistics: (α + β -1)/ se(α + β)
print(f"t-statistic for testing constant returns to scale : (α + β  -1)/ se(α + β) ={t_stat[0]:.4f}")
t_crit = stats.t.ppf(1-alpha/2, df=len(dat)-3) # Critical t-value for two-tailed test
print(f"Critial t-value for 95% for to sidet test: {t_crit:.4f}")
p_val = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=len(dat)-3)) # Two-tailed p-values for t-stats
print(f"p-value for testing constant returns to scale : {p_val[0]:.4f}")


t-statistic for testing constant returns to scale : (α + β  -1)/ se(α + β) =-3.6028
Critial t-value for 95% for to sidet test: 1.9604
p-value for testing constant returns to scale : 0.0003


### Test for constant returns to scale (test på een parameter)
Cobb-Douglas produktionsfunktion
$$
\log(Y_{it}) = a_0 + \alpha \log(L_{it}) + \beta \log(K_{it})  + u_{it} 
$$
Transformér modellen så vi kan lave hypotesetest med en parameter:
$$
\log(Y_{it}) = a_0 + \theta \log(L_{it}) + \beta (\log(K_{it})-\log(L_{it}))  + u_{it}$$
hvor $\theta = \alpha + \beta$

Vi tester hypothesen:
$$H_0: \theta = 1 \quad H_1: \theta \neq 1$$

Test størrelse
$$t=\frac{\hat{\theta}-1}{\sqrt{Var(\hat{\theta})}} \sim t(n-k)$$

### Let's code

In [7]:

# Estimer model med log(k/l) som eneste forklarende variabel
dat['log(cap/emp)'] = dat['lcap'] - dat['lemp'] # Create new variable log(k/l)
CobbDouglas2 = mlr.ols(dat[['const', 'lemp', 'log(cap/emp)']], dat['ldsa']) # estimate model using mlr
mlr.output(CobbDouglas2) # Print the summary using the mlr.summary() function

# t-statistics: \theta  -1 / se(\theta)
print('Coefficient for log(emp):',CobbDouglas2['beta_hat'][1])
t_stat2=(CobbDouglas2['beta_hat'][1] -1) / CobbDouglas2['standard_errors'][1] # t-statistics: θ -1 / se(θ)
print(f"t-statistic for testing constant returns to scale : (θ  -1) / se(θ) ={t_stat2[0]:.4f}")
t_crit2 = stats.t.ppf(1-alpha/2, df=len(dat)-3) # Critical t-value for two-tailed test
print(f"Critial t-value for 95% for to sidet test: {t_crit2:.4f}")
p_val2 = 2 * (1 - stats.t.cdf(np.abs(t_stat2), df=len(dat)-3)) # Two-tailed p-values for t-stats
print(f"p-value for testing constant returns to scale : {p_val2[0]:.4f}")

OLS Regression Results for Dependent Variable: ldsa
Number of Observations: 5292
Degrees of Freedom: 5289 (Residual), 3 (Model)
R-squared: 0.9139
TSS: 8037.3115, RSS: 691.9438, ESS: 7345.3677
Variable                Coefficient     Std. Error           t       P>|t|    95% Conf. Interval
--------------------------------------------------------------------------------------------------
const                        0.0000         0.0050      0.0000      1.0000   [ -0.0097, 0.0097  ]
lemp                         0.9848         0.0042    233.6604      0.0000   [  0.9766, 0.9931  ]
log(cap/emp)                 0.3100         0.0091     33.9237      0.0000   [  0.2921, 0.3280  ]
Coefficient for log(emp): [0.98481525]
t-statistic for testing constant returns to scale : (θ  -1) / se(θ) =-3.6028
Critial t-value for 95% for to sidet test: 1.9604
p-value for testing constant returns to scale : 0.0003


## Part 3: Test af multiple lineær hypoteser (F-test)
Tilbage til lønligningen

### Implementering af F-test

Vi estimerer:
$$
\log(\text{wage}_i) = \beta_0 + \beta_1 \text{educ}_i + \beta_2 \text{experience}_i + \beta_3 \text{experience}^2_i + \beta_4 \text{experience}^3_i + u_i
$$

For at teste restriktioner på $\beta$, udfører vi en **F-test** mellem en **fuld model** og en **restrikteret model**.

1. **F-statistik**:
   $$
   F = \frac{(RSS_r - RSS_{ur}) / q}{RSS_{ur} / (n - k_{ur})} \sim F(q, n - k_{ur})
   $$
   Hvor:
   - $RSS_r$: RSS for den restrikterede model
   - $RSS_{ur}$: RSS for den fulde model
   - $q$: antal restriktioner

2. **P-værdi**:
   P-værdien beregnes som:
   $$
   p = 1 - F_{\text{cdf}}(F_{\text{stat}}, q, n - k_{ur})
   $$
   hvor $F_{\text{cdf}}$ er kumulativ fordelingsfunktion for F-fordelingen, og $F_{\text{stat}}$ er den beregnede F-statistik.


In [9]:
y=df['lwage']
X_ur = df[['const', 'educ', 'experience', 'experience2','experience3']]
X_r = df[['const']]

def Ftest(y, X_ur, X_r):
    # Fit the unrestricted and unrestricted models and print summary output
    m_ur = mlr.ols(X_ur, y)   
    m_r = mlr.ols(X_r, y)
    mlr.summary([m_ur, m_r])

    # Number of observations (n) and parameters (k) in the unrestricted model
    n = m_ur['n']
    k_ur = m_ur['k'] # Number of columns in X (including intercept)
    k_r = m_r['k']   # Number of columns in X_r (including intercept)

    # Number of restrictions (q) - this is the difference in parameters between the models
    q = k_ur - k_r

    # Residual sum of squares (RSS) for both models
    RSS_ur = m_ur['RSS']
    RSS_r = m_r['RSS']

    # Compute the F-statistic
    F_stat = ((RSS_r - RSS_ur) / q) / (RSS_ur / (n - k_ur))

    # Compute the p-value based on the F-distribution
    p_value = 1 - stats.f.cdf(F_stat, q, n - k_ur)

    # Display the results
    print(f"\nF-test for multible lineære hypotester: ")
    print(f"   Number of restrictions: {q:d}")
    print(f"   Df, unrestricted model: {n -k_ur:d}")
    print(f"   Df, restricted model: {n - k_r:d}")
    print(f"   F-statistic: {F_stat:.4f} ~ F({q:d},{n - k_ur:d})")
    print(f"   P-value: {p_value:.4f}")

Ftest(y, X_ur, X_r)

                    Model 1  Model 2
Dependent variable    lwage    lwage
             const   4.2699   4.8834
                   (0.0466) (0.0098)
              educ   0.0270         
                   (0.0027)         
        experience   0.0428         
                   (0.0102)         
       experience2  -0.0018         
                   (0.0007)         
       experience3   0.0286         
                   (0.0154)         
         R_squared   0.2058  -0.0000
               TSS 111.2507 111.2507
               RSS  88.3507 111.2507
               ESS  22.9000  -0.0000
                 n     1078     1078

F-test for multible lineære hypotester: 
   Number of restrictions: 4
   Df, unrestricted model: 1073
   Df, restricted model: 1077
   F-statistic: 69.5288 ~ F(4,1073)
   P-value: 0.0000


In [11]:
X_r = X_ur.drop(columns=['educ'])  # Restrict the model by removing 'educ'
#X_r = X.drop(columns=['experience', 'experience2','experience3'])  # remove expericence altogether
#X_r = X.drop(columns=['experience2','experience3'])  # lnwage is linear in experience
Ftest(y, X_ur, X_r)

                    Model 1  Model 2
Dependent variable    lwage    lwage
             const   4.2699   4.5400
                   (0.0466) (0.0400)
        experience   0.0428   0.0536
                   (0.0102) (0.0106)
       experience2  -0.0018  -0.0025
                   (0.0007) (0.0008)
       experience3   0.0286   0.0413
                   (0.0154) (0.0161)
              educ   0.0270         
                   (0.0027)         
         R_squared   0.2058   0.1292
               TSS 111.2507 111.2507
               RSS  88.3507  96.8729
               ESS  22.9000  14.3778
                 n     1078     1078

F-test for multible lineære hypotester: 
   Number of restrictions: 1
   Df, unrestricted model: 1073
   Df, restricted model: 1074
   F-statistic: 103.4994 ~ F(1,1073)
   P-value: 0.0000
